In [3]:
import pyspark

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('dfml').getOrCreate()
spark

In [17]:
filepath = "file1.csv"
training = spark.read.csv(filepath, header = True, inferSchema = True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [18]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [19]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [25]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols = ['age', 'Experience'], outputCol = 'Independent Features')

In [26]:
output = featureAssembler.transform(training)

In [27]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [28]:
output.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Independent Features: vector (nullable = true)



In [29]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [30]:
data = output.select(['Independent Features', "Salary"])

In [31]:
data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [34]:
from pyspark.ml.regression import LinearRegression 
X_train , y_train = data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = "Independent Features", labelCol = "Salary")
regressor = regressor.fit(X_train)

In [38]:
y_pred = regressor.evaluate(X_train)

In [39]:
y_pred.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16169.179229480775|
|          [23.0,2.0]| 18000|17497.487437185944|
|          [24.0,3.0]| 20000|18797.319932998347|
|          [29.0,4.0]| 20000| 20211.05527638182|
|          [30.0,8.0]| 25000|25324.958123953118|
+--------------------+------+------------------+

